# **Token & Embedding 자세히 살펴보기**



---


- 💡 **NOTE**
    - 이 노트북의 코드를 실행하려면 GPU를 사용하는 것이 좋습니다. 구글 코랩에서는 **런타임 > 런타임 유형 변경 > 하드웨어 가속기 > T4 GPU**를 선택하세요.


---



# **Tokenization**

In [1]:
# Phi-3 모델과 호환성 때문에 transformers 4.48.3 버전을 사용합니다.
!pip install transformers==4.48.3

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 44.4/44.4 kB 3.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 9.7/9.7 MB 55.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.1/3.1 MB 69.1 MB/s eta 0:00:00
  Attempting uninstall: tokenizers
    Found existing installation: tokenizers 0.22.0
    Uninstalling tokenizers-0.22.0:
      Successfully uninstalled tokenizers-0.22.0
  Attempting uninstall: transformers
    Found existing installation: transformers 4.56.1
    Uninstalling transformers-4.56.1:
      Successfully uninstalled transformers-4.56.1


In [2]:
# 깃허브에서 위젯 상태 오류를 피하기 위해 진행 표시줄을 나타내지 않도록 설정합니다.
from transformers.utils import logging

logging.disable_progress_bar()

## LLM 토큰화 확인


**1. 모델 로드하고 토크나이저 지정하기**

In [3]:
from transformers import AutoModelForCausalLM, AutoTokenizer

# 모델과 토크나이저를 로드합니다.
model = AutoModelForCausalLM.from_pretrained(
    "microsoft/Phi-3-mini-4k-instruct",
    device_map="cuda",
    torch_dtype="auto",
    trust_remote_code=True,
)
tokenizer = AutoTokenizer.from_pretrained("microsoft/Phi-3-mini-4k-instruct")

/usr/local/lib/python3.12/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(
A new version of the following files was downloaded from https://huggingface.co/microsoft/Phi-3-mini-4k-instruct:
- configuration_phi3.py
. Make sure to double-check they do not contain any added malicious code. To avoid downloading new versions of the code file, you can pin a revision.
A new version of the following files was downloaded from https://huggingface.co/microsoft/Phi-3-mini-4k-instruct:
- modeling_phi3.py
. Make sure to double-check they do not contain any added mal

**2.입력 프롬프트를 토큰으로 나누기**

In [4]:
prompt = '''Write an email apologizing to Sarah for the tragic gardening mishap.
Explain how it happened.<|assistant|>'''

# 입력 프롬프트를 토큰으로 나눕니다.
input_ids = tokenizer(prompt, return_tensors="pt").input_ids.to("cuda")

# 텍스트를 생성합니다.
generation_output = model.generate(
  input_ids=input_ids,
  max_new_tokens=20   # 토큰을 20개 생성
)

# 출력을 프린트합니다.
print(tokenizer.decode(generation_output[0]))  # decode 메서드: 토큰ID를 실제 텍스트로 변환

The attention mask is not set and cannot be inferred from input because pad token is same as eos token. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
The `seen_tokens` attribute is deprecated and will be removed in v4.41. Use the `cache_position` model input instead.
`get_max_cache()` is deprecated for all Cache classes. Use `get_max_cache_shape()` instead. Calling `get_max_cache()` will raise error from v4.48


Write an email apologizing to Sarah for the tragic gardening mishap.
Explain how it happened.<|assistant|> Subject: Sincere Apologies for the Gardening Mishap

Dear Sarah


In [5]:
print(input_ids)

tensor([[14350,   385,  4876, 27746,  5281,   304, 19235,   363,   278, 25305,
           293, 16423,   292,   286,   728,   481, 29889,    13,  9544,  7420,
           920,   372,  9559, 29889, 32001]], device='cuda:0')


In [6]:
for id in input_ids[0]:
   print(tokenizer.decode(id))

Write
an
email
apolog
izing
to
Sarah
for
the
trag
ic
garden
ing
m
ish
ap
.


Exp
lain
how
it
happened
.
<|assistant|>


In [7]:
generation_output

tensor([[14350,   385,  4876, 27746,  5281,   304, 19235,   363,   278, 25305,
           293, 16423,   292,   286,   728,   481, 29889,    13,  9544,  7420,
           920,   372,  9559, 29889, 32001,  3323,   622, 29901,   317,  3742,
           406,  6225, 11763,   363,   278, 19906,   292,   341,   728,   481,
            13,    13, 29928,   799, 19235]], device='cuda:0')

In [8]:
print(tokenizer.decode(3323))
print(tokenizer.decode(622))
print(tokenizer.decode([3323, 622]))
print(tokenizer.decode(29901))

Sub
ject
Subject
:


## **토크나이저가 텍스트를 분할하는 방법**

1. 모델 설계시 모델 작성자가 토큰화 방법을 선택
    - GTP 모델 : BPE(byte pair encoding)
    - BERT 모델 : WordPiece
2. 토큰화 방법을 선택한 후에 어휘사전 크기와 특수 토큰 같은 토크나이저 설계상의 여러 가지 선택을 해야함
3. 토크나이저는 특정 데이터셋에서 훈련하여 해당 데이터셋을 표현하는 최상의 어휘사전을 구축해야 함

## **토큰 종류**

- **단어 토큰**
    - word2vec와 같은 초기 토큰화에 사용됨 --> 현재는 덜 사용됨
    - 지금도 추천 시스템과 같은 곳에서 사용
    - (단점: 훈련된 후에 데이터셋에 새롭게 추가된 단어는 처리할 수 없다.-->되도록 많은 어휘사전을 만들어야 한다.)

- **부분단어 토큰** (완전단어+부분완전단어 포함)
    - 새로운 단어를 (어휘사전에 포함되어 있을 가능성이 높은) 더 작은 단위로 나눈다.
    - **평균적으로 토큰당 세 개의 문자로 구성됨**
- **문자 토큰**
    - 대체할 원시 문자가 있기 때문에 새로운 단어를 잘 처리할 수 있다.
    - 토큰화는 쉽지만 모델링은 어렵다.--> 문자를 조합하는 정보까지 모델링해야함)
- **바이트 토큰**
    - 유니코드 문자를 표현하는 바이트로 토큰을 분할하는 방법
    - '토큰화-프리 인코딩' 라고 부름
    - 다국어 환경에서 경쟁력이 있다고 봄

## **훈련된 LLM 토큰나이저 비교하기**


- **중점 비교 요소**
    - **토큰화 방법** :  BEP, SentencePiece, WordPiece
    - **토크나이저 파라미터**
        - 어휘사전 크기 : 토크나이저가 어휘사전에 얼마나 많은 토큰을 포함할 건가?
        - 특수 토큰 : 모델이 추적해야할 특수 토큰은 무엇인가?
        - 대소문자 : 영어와 같은 대소문자를 어떻게 다뤄야 할까?

In [9]:
from transformers import AutoModelForCausalLM, AutoTokenizer

colors_list = [
    '102;194;165', '252;141;98', '141;160;203',
    '231;138;195', '166;216;84', '255;217;47'
]

def show_tokens(sentence, tokenizer_name):
    tokenizer = AutoTokenizer.from_pretrained(tokenizer_name)
    token_ids = tokenizer(sentence).input_ids
    for idx, t in enumerate(token_ids):
        # 텍스트를 인코딩한 후 다시 디코딩했을 때 원본 텍스트와 동일해지려면
        # clean_up_tokenization_spaces를 False로 지정해야 합니다.
        # 현재 이 매개변수의 기본값은 None(True에 해당)이며
        # transformers 4.45에서 True로 바뀔 예정입니다.
        # https://github.com/huggingface/transformers/issues/31884
        print(
            f'\x1b[0;30;48;2;{colors_list[idx % len(colors_list)]}m' +
            tokenizer.decode(t, clean_up_tokenization_spaces=False) +
            '\x1b[0m',
            end=' '
        )

In [10]:
text = """
English and CAPITALIZATION
🎵 鸟
show_tokens False None elif == >= else: two tabs:"		" four spaces:"    "
12.0*50=600
"""

- **BERT 베이스 모델(uncased)(2018)**
    - 토큰화 방법 : WordPiece
    - 어휘사전 크기 : 30, 522
    - 특수 토큰:
        - [UNK] unk_token : 토크나이저가 인코딩 방법을 모르는 토큰
        - [SEP] sep_toketn :  특정 작업에서 두 개의 텍스트를 구분하기 위한 토큰(cross-endcoder)
        - [PAD] pad_token :  모델 입력에서 사용되지 않는 위치를 채우기 위한 패딩 토큰 --> 모델은 특정 길이(문맥 크기)의 입력을 기대하기 때문
        - [CLS] cls_token :  분류 작업을 위한 특수 토큰
        - [MASK] mask_token : 훈련 과정 동안 일부 토큰을 감추기 위해 사용되는 마스킹 토큰
    - 특징 : 줄 바꿈 인코딩 정보를 알지 못함

In [11]:
show_tokens(text, "bert-base-uncased")  # Uncased : 대소문자를 구분하지 않음

[CLS] english and capital ##ization [UNK] [UNK] show _ token ##s false none eli ##f = = > = else : two tab ##s : " " four spaces : " " 12 . 0 * 50 = 600 [SEP] 

- **BERT 베이스 모델(cased)(2018)**
    - 토큰화 방법 : WordPiece
    - 어휘사전 크기 : 28,996
    - 특수 토큰: uncased 버전과 동일

In [12]:
show_tokens(text, "bert-base-cased")

[CLS] English and CA ##PI ##TA ##L ##I ##Z ##AT ##ION [UNK] [UNK] show _ token ##s F ##als ##e None el ##if = = > = else : two ta ##bs : " " four spaces : " " 12 . 0 * 50 = 600 [SEP] 

- **GPT-2(2019)**
    - 토큰화 방법 : BPE
    - 어휘사전 크기 : 50,257
    - 특수 토큰:
        - <|endoftext|>
    - 특징: 줄바꿈이 토크나이저 내에서 표현됨

In [13]:
show_tokens(text, "gpt2")


 English  and  CAP ITAL IZ ATION 
 � � �  � � � 
 show _ t ok ens  False  None  el if  ==  >=  else :  two  tabs :" 	 	 "  four  spaces :"        " 
 12 . 0 * 50 = 600 
 

- **Flan-T5(2022)**
    - 토큰화 방법 : SentencePiece
    - 어휘사전 크기 : 32,100
    - 특수 토큰:
        - `<unk>` unk_token
        - `<pad>` pad_token
    - 특징 :
        - 줄바꿈이나 공백 토큰이 없음 --> 모델이 코드를 다루기 어렵다.
        - 이모자와 한자가 모두 <unk> 토큰으로 바꾸었음 --> 모델이 이런 토큰을 식별하지 못함

In [14]:
show_tokens(text, "google/flan-t5-small")

English and CA PI TAL IZ ATION  <unk>  <unk> show _ to ken s Fal s e None  e l if = = > = else : two tab s : " " four spaces : " " 12. 0 * 50 = 600  </s> 

- **GPT-4(2023)**
    - 토큰화 방법 : BPE
    - 어휘사전 크기 : 100,000 이상
    - 특수 토큰:
        - `<endoftext>`
        - 중간 토큰을 채우도록 훈련됨. 세 개의 특수 토큰을 사용해 앞, 뒤에 나오는 텍스트를 고려해 LLM이 완성된 문장을 생성함
            - <|fim_prefix|>
            - <|fim_middle|>
            - <|fim_suffix|>
    - 특징 :
        - GPT-2 토크나이저와 비슷하게 동작
        - GPT-4는 4개의 공백을 하나의 토큰으로 표현
        - elif를 하나의 토큰으로 표현 --> 자연어외에 코드에 초점을 맞추고 있음
        - 더 적은 토큰을 사용해 대부분의 단어를 표현함(CAPITALIZATION 두 개의 토큰)

In [15]:
# 공식 토크나이저는 `tiktoken`이지만 허깅 페이스 플랫폼에 동일한 토크나이저가 있습니다.
show_tokens(text, "Xenova/gpt-4")


 English  and  CAPITAL IZATION 
 � � �  � � � 
 show _tokens  False  None  elif  ==  >=  else :  two  tabs :" 	 	 "  four  spaces :"      "
 12 . 0 * 50 = 600 
 

- **StarCoder2(2024)**
    - 코드 생성에 초점을 맞춘 150개의 파라미터를 가진 디코더 모델
    - 토큰화 방법 : BPE
    - 어휘사전 크기 : 49,152
    - 특수 토큰:
        - `<endoftext>`
        - 중간 채우기를 위한 토큰: `<fim_prefix>`, `<fim_middle>`, `<fim_suffix>`, `<fim_pad>`
        - `<filename>`, `<reponame>`, `<gh_stars>`
    - 특징 :
        - 코드를 표현할 때 문맥 관리가 중요(예를 들어 파일에서 다른 파일에 정의된 함수를 호출하는 경우)
        - 모델은 같은 저장소의 다른 파일에 있는 코드를 식별하고 다른 저장소에 있는 코드와 구분할 수 있어야함

In [16]:
show_tokens(text, "bigcode/starcoder2-15b")


 English  and  CAPITAL IZATION 
 � � �   � � 
 show _ tokens  False  None  elif  ==  >=  else :  two  tabs :" 	 	 "  four  spaces :"      " 
 1 2 . 0 * 5 0 = 6 0 0 
 

- **Galatica**

    - 과학 지식에 초점을 맞추어 많은 과학 논문, 참고 자료, 지식 데이터에서 훈련됨.
    - 토큰화에 더 주의를 기울여 데이터셋에 있는 뉘앙스에 민감함 --> 인요, 추론, 수학, 펩타이드 서열, DNA 서열을 위한 특수 토큰이 있음
    -
    - 토큰화 방법 : BPE
    - 어휘사전 크기 : 50,000
    - 특수 토큰:
        - `<s>`
        - `<pad>`
        - `</s>`,
        - `<unk>`
        - 참조/인용은 [START_REF]와 [END_REF]로 감쌈
        - 단계별 추론: <work>는 모델이 CoT(chain-of-thought)추론에 사용하는 토큰
    - 특징 :
        - 코드를 염두에 둠, StarCoder2와 비슷하게 동작
        - 탭도 하나의 토큰으로 인코딩

In [17]:
show_tokens(text, "facebook/galactica-1.3b")


 English  and  CAP ITAL IZATION 
 � � � �  � � � 
 show _ tokens  False  None  elif   ==   > =  else :  two  t abs : " 		 "  four  spaces : "      " 
 1 2 . 0 * 5 0 = 6 0 0 
 

- **Phi-3(Llama 2)**
    - 여러 갱의 특수 토큰을 추가한 Llama 2 토크나이저 재사용
    -
    - 토큰화 방법 : BPE
    - 어휘사전 크기 : 32,000
    - 특수 토큰:
        - `<|endoftext|>`
        - 채팅토큰
            - `<|user|>`,
            - `<|assistant|>`
            - `<|system|>`
    - 특징 :
        - 채팅에 초점을 맞춤

In [18]:
show_tokens(text, "microsoft/Phi-3-mini-4k-instruct")

 
 English and C AP IT AL IZ ATION 
 � � � �  � � � 
 show _ to kens False None elif == >= else : two tabs :" 	 	 " four spaces :"    " 
 1 2 . 0 * 5 0 = 6 0 0 
 



---



## **한국어 특화된 토크나이저**

- **한글의 특수성**:
    - **교착어**: 조사가 붙어 단어 형태가 무한대로 변형 ("학교", "학교가", "학교에서", "학교로부터"...)
    - **띄어쓰기 불규칙**: SNS, 댓글 등에서 띄어쓰기가 일관되지 않음
    - **자모 조합**: 초성+중성+종성이 결합되어 하나의 음절 형성 (ex: ㄱ+ㅏ+ㅁ = 감)

- 발전 과정:
    - 2013-2016: KoNLPy, 형태소 분석기 (Mecab, Okt) 등장
    - 2018: 다국어 BERT에 한국어 포함되었으나 성능 부족
    - 2020: KoBERT (SKT) 출시 - 한국어 Wikipedia로 학습
    - 2021: KoGPT, KoBART 등 한국어 특화 모델 등장
    - 2022-2024: KoAlpaca, Polyglot-Ko, KULLM 등 다양한 한국어 LLM 개발

- 주요 참고자료:
    - SKT KoBERT: https://github.com/SKTBrain/KoBERT
    - Kakao KoGPT: https://github.com/kakaobrain/kogpt

### **[미션]** 한국어 LLM 찾아 토큰화 해보기.
- 허깅페이스에서 사전학습된 한국어 LLM을 여러 개(2개 이상) 찾고 위와 같이 토큰화 & 비교해보세요.

In [23]:
ko_text = """
    안녕하세요! 저는 인공지능을 공부하고 있습니다.,
    한글은 세종대왕이 창제하신 문자입니다. 😊,
    카카오와 네이버는 한국의 대표적인 IT 기업입니다.,
    자연어처리(NLP)는 컴퓨터가 인간의 언어를 이해하도록 하는 기술이다.,
"""

# 예제 : SKT의 KoBERT - 비교적 안정적
print(ko_text)
show_tokens(ko_text, "skt/kobert-base-v1")


    안녕하세요! 저는 인공지능을 공부하고 있습니다.,
    한글은 세종대왕이 창제하신 문자입니다. 😊,
    카카오와 네이버는 한국의 대표적인 IT 기업입니다.,
    자연어처리(NLP)는 컴퓨터가 인간의 언어를 이해하도록 하는 기술이다.,

 ᄋ [UNK] ᄒ [UNK] ᄋ [UNK] !  [UNK]  ᄋ ᅵ [UNK] ᄀ [UNK] ᅵ [UNK] ᄋ [UNK]  ᄀ [UNK] ᄒ [UNK] ᄀ [UNK]  ᄋ ᅵ [UNK] ᅵ [UNK] . ,  ᄒ [UNK] ᄀ [UNK] ᄋ [UNK]  [UNK] ᄋ [UNK] ᄋ ᅵ  [UNK] ᄒ [UNK] ᅵ [UNK]  [UNK] ᄋ ᅵ [UNK] ᅵ [UNK] .  [UNK] ,  ᄏ [UNK] ᄏ [UNK] ᄋ [UNK] ᄋ [UNK]  [UNK] ᄋ ᅵ [UNK]  ᄒ [UNK] ᄀ [UNK] ᄋ [UNK]  [UNK] ᄋ ᅵ [UNK] IT  ᄀ ᅵ ᄋ [UNK] ᄋ ᅵ [UNK] ᅵ [UNK] . ,  [UNK] ᄋ [UNK] ᄋ [UNK] ᅵ ( N L P ) [UNK]  ᄏ [UNK] ᄀ [UNK]  ᄋ ᅵ [UNK] ᄀ [UNK] ᄋ [UNK]  ᄋ [UNK] ᄋ [UNK]  ᄋ ᅵ ᄒ [UNK] ᄒ [UNK]  ᄒ [UNK]  ᄀ ᅵ [UNK] ᄋ ᅵ [UNK] . ,  [UNK] [UNK] 

In [24]:
# EleutherAI의 Polyglot-Ko 시리즈

print(ko_text)
show_tokens(ko_text, "EleutherAI/polyglot-ko-1.3b")


    안녕하세요! 저는 인공지능을 공부하고 있습니다.,
    한글은 세종대왕이 창제하신 문자입니다. 😊,
    카카오와 네이버는 한국의 대표적인 IT 기업입니다.,
    자연어처리(NLP)는 컴퓨터가 인간의 언어를 이해하도록 하는 기술이다.,


        안녕 하 세요 !  저 는  인공지능 을  공부 하고  있 습니다 . , 
        한글 은  세종 대왕 이  창 제 하 신  문자 입니다 .   � � � � , 
        카카오 와  네이버 는  한국 의  대표 적 인  IT  기업 입니다 . , 
        자연 어 처리 ( NL P ) 는  컴퓨터 가  인간 의  언어 를  이해 하 도록  하 는  기술 이 다 . , 
 

In [25]:
# Beomi의 KcBERT - 댓글 데이터 특화

print(ko_text)
show_tokens(ko_text, "beomi/kcbert-base")


    안녕하세요! 저는 인공지능을 공부하고 있습니다.,
    한글은 세종대왕이 창제하신 문자입니다. 😊,
    카카오와 네이버는 한국의 대표적인 IT 기업입니다.,
    자연어처리(NLP)는 컴퓨터가 인간의 언어를 이해하도록 하는 기술이다.,

[CLS] 안녕 ##하세요 ! 저는 인공 ##지능 ##을 공부하고 있습니다 . , 한글 ##은 세종대왕 ##이 창 ##제 ##하신 문자 ##입니다 . 😊 , 카카오 ##와 네이버는 한국의 대표적인 IT 기업 ##입니다 . , 자연 ##어 ##처리 ( N ##L ##P ) 는 컴퓨터 ##가 인간의 언어 ##를 이해 ##하도록 하는 기술이 ##다 . , [SEP] 

In [27]:
# monologg의 KoBERT (Transformers 호환)

print(ko_text)
show_tokens(ko_text, "monologg/kobert-lm")


    안녕하세요! 저는 인공지능을 공부하고 있습니다.,
    한글은 세종대왕이 창제하신 문자입니다. 😊,
    카카오와 네이버는 한국의 대표적인 IT 기업입니다.,
    자연어처리(NLP)는 컴퓨터가 인간의 언어를 이해하도록 하는 기술이다.,

The repository for monologg/kobert-lm contains custom code which must be executed to correctly load the model. You can inspect the repository content at https://hf.co/monologg/kobert-lm.
You can avoid this prompt in future by passing the argument `trust_remote_code=True`.

Do you wish to run the custom code? [y/N] y


A new version of the following files was downloaded from https://huggingface.co/monologg/kobert-lm:
- tokenization_kobert.py
. Make sure to double-check they do not contain any added malicious code. To avoid downloading new versions of the code file, you can pin a revision.


[CLS] 안 녕 하세요 ! 저 는 인 공 지 능 을 공부 하고 있습니다 . , 한 글 은 세종 대 왕 이 창 제 하 신 문자 입니다 .  [UNK] , 카카오 와 네이버 는 한국의 대표적인 IT 기업 입니다 . , 자연 어 처리 ( N L P ) 는 컴퓨터 가 인간 의 언 어 를 이해 하도록 하는 기술 이다 . , [SEP] 

In [29]:
# SKT KoGPT2 (생성형 GPT 계열)

print(ko_text)
show_tokens(ko_text, "skt/kogpt2-base-v2")


    안녕하세요! 저는 인공지능을 공부하고 있습니다.,
    한글은 세종대왕이 창제하신 문자입니다. 😊,
    카카오와 네이버는 한국의 대표적인 IT 기업입니다.,
    자연어처리(NLP)는 컴퓨터가 인간의 언어를 이해하도록 하는 기술이다.,


    안녕 하 세 요 ! 저 는 인공 지 능을 공부하고 있습니다. , 
    한글 은 세종 대 왕이 창제 하신 문자 입니다.  😊 , 
    카 카 오와 네이 버 는 한국의 대표적인 IT 기업 입니다. , 
    자연 어 처 리( N L P )는 컴퓨터가 인간의 언어를 이해 하도록 하는 기술 이다. , 
 

In [30]:
# KLUE RoBERTa (범용 한국어 RoBERTa)

print(ko_text)
show_tokens(ko_text, "klue/roberta-base")


    안녕하세요! 저는 인공지능을 공부하고 있습니다.,
    한글은 세종대왕이 창제하신 문자입니다. 😊,
    카카오와 네이버는 한국의 대표적인 IT 기업입니다.,
    자연어처리(NLP)는 컴퓨터가 인간의 언어를 이해하도록 하는 기술이다.,

[CLS] 안녕 ##하 ##세요 ! 저 ##는 인공지능 ##을 공부 ##하고 있 ##습 ##니다 . , 한글 ##은 세종 ##대왕 ##이 창 ##제 ##하 ##신 문자 ##입니다 . [UNK] , 카카오 ##와 네이버 ##는 한국 ##의 대표 ##적인 IT 기업 ##입니다 . , 자연 ##어 ##처리 ( N ##L ##P ) 는 컴퓨터 ##가 인간 ##의 언어 ##를 이해 ##하 ##도록 하 ##는 기술 ##이다 . , [SEP] 



---



# **토큰 임베딩(Token Embedding)**

- **언어** --> **토큰의 시퀀스**
- **충분히 좋은 모델**을 충분히 **큰 토큰 집합에서 훈련**한다면 **훈련 데이터셋에 있는 복잡한 패턴을 포착**하기 시작한다.
- **Embedding** --> 수치표현, **언어에 있는 의미와 패턴을 포착하기 위한 수치 표현 공간**

- 토크나이저가 초기화되고 훈련되고 나면 이를 사용해 언어 모델을 훈련함
    - 사전 훈련된 언어 모델이 해당 토크나이저와 연결되는 이유
    - 모델을 재훈련하지 않고는 다른 토크나이저를 사용할 수 없음

## (BERT와 같은)**언어 모델로 문맥을 고려한 단어 임베딩 만들기**

In [1]:
from transformers import AutoModel, AutoTokenizer

# 토크나이저를 로드합니다.
tokenizer = AutoTokenizer.from_pretrained("microsoft/deberta-v3-base")

# 언어 모델을 로드합니다.
model = AutoModel.from_pretrained("microsoft/deberta-v3-xsmall")

# 문장을 토큰으로 나눕니다.
tokens = tokenizer('Hello world', return_tensors='pt')

# 토큰을 처리합니다.
output = model(**tokens)[0]

/usr/local/lib/python3.12/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/52.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/579 [00:00<?, ?B/s]

spm.model:   0%|          | 0.00/2.46M [00:00<?, ?B/s]

/usr/local/lib/python3.12/dist-packages/transformers/convert_slow_tokenizer.py:564: UserWarning: The sentencepiece tokenizer that you are converting to a fast tokenizer uses the byte fallback option which is not implemented in the fast tokenizers. In practice this means that the fast version of the tokenizer can produce unknown tokens whereas the sentencepiece version would have converted these unknown tokens into a sequence of byte tokens matching the original piece of text.
  warnings.warn(


config.json:   0%|          | 0.00/578 [00:00<?, ?B/s]

pytorch_model.bin:   0%|          | 0.00/241M [00:00<?, ?B/s]

In [2]:
output.shape

torch.Size([1, 4, 384])

In [3]:
for token in tokens['input_ids'][0]:
    print(tokenizer.decode(token))

[CLS]
Hello
world
[SEP]


In [4]:
output

tensor([[[-3.3060, -0.0507, -0.1098,  ..., -0.1704, -0.1618,  0.6932],
         [ 0.8918,  0.0740, -0.1583,  ...,  0.1869,  1.4760,  0.0751],
         [ 0.0871,  0.6364, -0.3050,  ...,  0.4729, -0.1829,  1.0157],
         [-3.1624, -0.1436, -0.0941,  ..., -0.0290, -0.1265,  0.7954]]],
       grad_fn=<NativeLayerNormBackward0>)

## **텍스트 임베딩** (문장과 전체 문서)

- 텍스트 임베딩이란 하나의 벡터로 토큰보다 긴 텍스트를 표현하는 것
- **텍스트 임베딩 모델**은 텍스트 조각을 입력받아 텍스트를 표현하고 유영한 형태로 의미를 포착하는 하나의 벡터를 만드는 모델

In [5]:
from sentence_transformers import SentenceTransformer

# 모델을 로드합니다.
model = SentenceTransformer('sentence-transformers/all-mpnet-base-v2')

# 텍스트를 텍스트 임베딩으로 변환합니다.
vector = model.encode("Best movie ever!")

modules.json:   0%|          | 0.00/349 [00:00<?, ?B/s]

config_sentence_transformers.json:   0%|          | 0.00/116 [00:00<?, ?B/s]

README.md: 0.00B [00:00, ?B/s]

sentence_bert_config.json:   0%|          | 0.00/53.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/571 [00:00<?, ?B/s]

...

tokenizer_config.json:   0%|          | 0.00/363 [00:00<?, ?B/s]

vocab.txt: 0.00B [00:00, ?B/s]

tokenizer.json: 0.00B [00:00, ?B/s]

special_tokens_map.json:   0%|          | 0.00/239 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/190 [00:00<?, ?B/s]

In [6]:
vector.shape

(768,)

## **임베딩으로 노래 추천하기**

- 데이터셋: 코넬대학교 슈오첸이 모은 데이터셋(미국 전역에 있는 수백개의 라이도 방송국에서 가져온 재생목록)
    - 노래 재생목록 :
        - https://storage.googleapis.com/maps-premium/dataset/yes_complete/train.txt
    - 노래 메타데이터: (제목, 아티스트)
        - https://storage.googleapis.com/maps-premium/dataset/yes_complete/song_hash.txt


In [ ]:
!pip install gensim

- 데이터셋 로드하기

In [2]:
import pandas as pd
from urllib import request

# 재생목록 데이터셋 파일을 가져옵니다.
data = request.urlopen('https://storage.googleapis.com/maps-premium/dataset/yes_complete/train.txt')

# 재생목록 파일을 파싱합니다. 처음 두 줄은 메타데이터만 담고 있으므로 건너뜁니다.
lines = data.read().decode("utf-8").split('\n')[2:]
print(len(lines))

# 하나의 노래만 있는 재생목록은 삭제합니다.
playlists = [s.rstrip().split() for s in lines if len(s.split()) > 1]

# 노래의 메타데이터를 로드합니다.
songs_file = request.urlopen('https://storage.googleapis.com/maps-premium/dataset/yes_complete/song_hash.txt')
songs_file = songs_file.read().decode("utf-8").split('\n')
songs = [s.rstrip().split('\t') for s in songs_file]
songs_df = pd.DataFrame(data=songs, columns = ['id', 'title', 'artist'])
songs_df = songs_df.set_index('id')

11138


In [3]:
print('재생목록 #1:\n ', playlists[0], '\n')
print('재생목록 #2:\n ', playlists[1])

재생목록 #1:
  ['0', '1', '2', '3', '4', '5', '6', '7', '8', '9', '10', '11', '12', '13', '14', '15', '16', '17', '18', '19', '20', '21', '22', '23', '24', '25', '26', '27', '28', '29', '30', '31', '32', '33', '34', '35', '36', '37', '38', '39', '40', '41', '2', '42', '43', '44', '45', '46', '47', '48', '20', '49', '8', '50', '51', '52', '53', '54', '55', '56', '57', '25', '58', '59', '60', '61', '62', '3', '63', '64', '65', '66', '46', '47', '67', '2', '48', '68', '69', '70', '57', '50', '71', '72', '53', '73', '25', '74', '59', '20', '46', '75', '76', '77', '59', '20', '43'] 

재생목록 #2:
  ['78', '79', '80', '3', '62', '81', '14', '82', '48', '83', '84', '17', '85', '86', '87', '88', '74', '89', '90', '91', '4', '73', '62', '92', '17', '53', '59', '93', '94', '51', '50', '27', '95', '48', '96', '97', '98', '99', '100', '57', '101', '102', '25', '103', '3', '104', '105', '106', '107', '47', '108', '109', '110', '111', '112', '113', '25', '63', '62', '114', '115', '84', '116', '117', '118', 

- Word2Vec 훈련하기
    - 결과물: 각 노래에 대해 계산된 임베딩 결과 --> 이 임베딩으로 비슷한 노래를 찾을 수 있다.)

In [4]:
from gensim.models import Word2Vec

# Word2Vec 모델을 훈련합니다.
model = Word2Vec(
    playlists, vector_size=32, window=20, negative=50, min_count=1, workers=4
)

In [5]:
song_id = 2172

# 노래 ID 2172와 비슷한 노래를 찾으라고 모델에게 요청합니다.
model.wv.most_similar(positive=str(song_id))

[('2849', 0.9989044070243835),
 ('2976', 0.9971297979354858),
 ('3119', 0.9970974326133728),
 ('10072', 0.9968254566192627),
 ('2704', 0.9963011145591736),
 ('11522', 0.9962957501411438),
 ('3167', 0.9962912201881409),
 ('1922', 0.9962764382362366),
 ('3096', 0.9962455034255981),
 ('6624', 0.9961906671524048)]

In [6]:
print(songs_df.iloc[2172])

title     Fade To Black
artist        Metallica
Name: 2172 , dtype: object


In [7]:
import numpy as np

def print_recommendations(song_id):
    similar_songs = np.array(
        model.wv.most_similar(positive=str(song_id),topn=5)
    )[:,0]
    return  songs_df.iloc[similar_songs]

# 추천 노래 출력
print_recommendations(2172)

,title,artist
id,,
2849,Run To The Hills,Iron Maiden
2976,I Don't Know,Ozzy Osbourne
3119,There's Only One Way To Rock,Sammy Hagar
10072,Jailbreak,AC\/DC
2704,Over The Mountain,Ozzy Osbourne


In [8]:
print_recommendations(842)

,title,artist
id,,
5668,How We Do (w\/ 50 Cent),The Game
5816,Got Money (w\/ T-Pain),Lil Wayne
9666,Gimme The Light,Sean Paul
12205,Give It Up To Me,Sean Paul
331,Get Low (w\/ Ying Yang Twins),Lil' Jon & The Eastside Boyz
